In [14]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [15]:
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [16]:
# data import
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')
subs_df  = pd.read_csv('./data/submission.csv')

In [17]:
print(train_df.head())

   Id County Province_State Country_Region  Population    Weight        Date  \
0   1    NaN            NaN    Afghanistan    27657145  0.058359  2020-01-23   
1   2    NaN            NaN    Afghanistan    27657145  0.583587  2020-01-23   
2   3    NaN            NaN    Afghanistan    27657145  0.058359  2020-01-24   
3   4    NaN            NaN    Afghanistan    27657145  0.583587  2020-01-24   
4   5    NaN            NaN    Afghanistan    27657145  0.058359  2020-01-25   

           Target  TargetValue  
0  ConfirmedCases          0.0  
1      Fatalities          0.0  
2  ConfirmedCases          0.0  
3      Fatalities          0.0  
4  ConfirmedCases          0.0  


### opmerkingen

- De eerste kolom is te verwaarlozen omdat het alleen zorgt voor een unique identifier voor die regel.


- De kolommen ['County', 'Province_State', 'Country_Region'] geeft samen de locatie, omdat deze locaties in deze dataset geen relatie tot elkaar hebben kunnen we hier een gecombineerde locatie_id van maken


- Target kan 1 van de volgende 2 waarde bevatten ['ConfirmedCases', 'Fatalities']


- TargetValue is een maatstaf om Weight van landen met verschillende populatie getallen met elkaar te kunnen vergelijken. Het drukt het aantal 'Fatalities' & 'COnfirmedCases' uit in een percentage van de populatie

## Location restructure

Voor de locaties gelden dat er in de dataset met verschillende granulatiteit word gewerkt, zowel 
1. USA
2. USA, New York
3. USA , New York, New York 

komt voor in de dataset.

Dit zorgt ervoor dat als we USA selecteren we 3x dezelfde data selecteren.<br>
We willen zo min mogelijk knippen in de dataset dus voegen we een kolom toe om makkelijk een subselectie te kunnen maken


In [18]:
# replace location-columns by unique indentifiers 
train_df['level'] = np.where(pd.notnull(train_df[['County']]), 3, 1)
train_df['level'] = np.where(pd.notnull(train_df[['Province_State']]), 2)
train_df = train_df.replace(np.nan, '', regex=True)


# train_df['location_combined'] = train_df['Country_Region'] + "-" + train_df['Province_State'] + "-" + train_df['County']
# train_df = train_df.drop(['Province_State','County','Country_Region','Id'], axis=1)

ValueError: either both or neither of x and y should be given

In [ ]:
train_df.head()

In [ ]:
level_filter = train_df["level"]==2

train_df.where(level_filter)

## Date restructure

Om landen in de tijd te kunnen analyseren moeten we zorgen dat de dagen de juiste schaal hebben.

Dit betekend dat we geen string maar een int gaan gebruiken. we gebruiken de eerste dag en de laatste dag die voorkomt in de dataset om te bepalen wat de range word, hier loop-en we doorheen om te kijken welke dagen voorkomen in de dataset. Met deze dagen bouwen we een dict op om de nieuwe index toe te passen 

> nergens word gehardcode zodat wanneer de dataset meer datums bevat de code blijft werken

> we gaan ervanuit dat er geen datums tussen de eerste dag en de laatste dag bestaan die in de dataset missen

In [19]:
# - verwijderen om als int te kunnen casten
train_df["Date"] = train_df["Date"].replace({'-': ''}, regex=True)

In [20]:
# kolom casten naar int

train_df["Date"] = pd.to_numeric(train_df["Date"])

In [21]:
# wat is de range van datums

min_date = train_df['Date'].min()
max_date = train_df['Date'].max()
print(min_date)
print(max_date)

20200123
20200518


In [22]:
dict_values = ""
custom_index = 0

for i in range(min_date, (max_date+1)):
    
    # komt het getal in de dataset voor?
    if int(i) in train_df['Date'].unique():
        
        # ja, voeg een entry toe aan de dict <   20200101: 1,   >
        dict_values += str(i) + ": " + str(custom_index) + ", "
        custom_index+=1
        
exec("date_dict = {" + dict_values + "}")

In [23]:
print(date_dict)

{20200123: 0, 20200124: 1, 20200125: 2, 20200126: 3, 20200127: 4, 20200128: 5, 20200129: 6, 20200130: 7, 20200131: 8, 20200201: 9, 20200202: 10, 20200203: 11, 20200204: 12, 20200205: 13, 20200206: 14, 20200207: 15, 20200208: 16, 20200209: 17, 20200210: 18, 20200211: 19, 20200212: 20, 20200213: 21, 20200214: 22, 20200215: 23, 20200216: 24, 20200217: 25, 20200218: 26, 20200219: 27, 20200220: 28, 20200221: 29, 20200222: 30, 20200223: 31, 20200224: 32, 20200225: 33, 20200226: 34, 20200227: 35, 20200228: 36, 20200229: 37, 20200301: 38, 20200302: 39, 20200303: 40, 20200304: 41, 20200305: 42, 20200306: 43, 20200307: 44, 20200308: 45, 20200309: 46, 20200310: 47, 20200311: 48, 20200312: 49, 20200313: 50, 20200314: 51, 20200315: 52, 20200316: 53, 20200317: 54, 20200318: 55, 20200319: 56, 20200320: 57, 20200321: 58, 20200322: 59, 20200323: 60, 20200324: 61, 20200325: 62, 20200326: 63, 20200327: 64, 20200328: 65, 20200329: 66, 20200330: 67, 20200331: 68, 20200401: 69, 20200402: 70, 20200403: 71, 2

In [ ]:
train_df['Date'] = train_df['Date'].replace(date_dict)

In [9]:
unique_locations = train_df.location_id.unique()
unique_targets = train_df.Target.unique()

print("There are "+str(len(unique_locations))+"\t unique locations")
print("There are "+str(len(unique_targets))+"\t unique Targets")


AttributeError: 'DataFrame' object has no attribute 'location_id'

In [10]:
train_df

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue,level
0,1,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-23,ConfirmedCases,0.0,1
1,2,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-23,Fatalities,0.0,1
2,3,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-24,ConfirmedCases,0.0,1
3,4,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-24,Fatalities,0.0,1
4,5,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-25,ConfirmedCases,0.0,1
...,...,...,...,...,...,...,...,...,...,...
810337,969590,NaN,NaN,Zimbabwe,14240168,0.607106,2020-05-16,Fatalities,0.0,1
810338,969591,NaN,NaN,Zimbabwe,14240168,0.060711,2020-05-17,ConfirmedCases,2.0,1
810339,969592,NaN,NaN,Zimbabwe,14240168,0.607106,2020-05-17,Fatalities,0.0,1
810340,969593,NaN,NaN,Zimbabwe,14240168,0.060711,2020-05-18,ConfirmedCases,2.0,1


In [ ]:
for loc in unique_locations:
    for tar in unique_targets:
        
        filter1 = train_df["Target"]==tar
        filter2 = train_df["location_id"]==loc
        
        temp = train_df.where(filter1)
        temp = train_df.where(filter2)
        
        
        continue
